In [3]:
print("This is jupyter notebook running inside a virtual environment")

This is jupyter notebook running inside a virtual environment


In [1]:
import openai

In [1]:
pip show transformers

Name: transformers
Version: 4.41.2
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /Users/shreyashprashu/.pyenv/versions/3.8.10/envs/mypyvenvenv38/lib/python3.8/site-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [1]:
from transformers.utils import logging
logging.set_verbosity_error()

In [2]:
from transformers import pipeline

2024-06-10 00:31:02.909721: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
chatbot = pipeline(task="conversational",
                   model="facebook/blenderbot-400M-distill")

pytorch_model.bin:   0%|          | 0.00/730M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/347 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/127k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/62.9k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/16.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/310k [00:00<?, ?B/s]

In [10]:
user_message = """
What are some fun activities I can do on a rainy day?
"""

In [11]:
from transformers import Conversation

In [12]:
conversation = Conversation(user_message)

In [13]:
print(conversation)

Conversation id: bd4229aa-3233-43e6-b346-5f3a64c4405f
user: 
What are some fun activities I can do on a rainy day?




In [14]:
conversation = chatbot(conversation)

In [15]:
print(conversation)

Conversation id: bd4229aa-3233-43e6-b346-5f3a64c4405f
user: 
What are some fun activities I can do on a rainy day?

assistant:  I like to go to the beach on rainy days. It's so relaxing.

